In [138]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('MyApp').getOrCreate()

In [139]:
"""
Loading the training csv.
"""

training = spark.read.option("escape", "\"").csv('../datasets/imdb/train_movies_extra_data_new.csv', header=True, inferSchema=True)

In [140]:
"""
Loading the validation csv.
"""

validation = spark.read.option("escape", "\"").csv('../datasets/imdb/validation_extra_data.csv', header=True, inferSchema=True)

In [141]:
"""
Loading the test csv.
"""

test = spark.read.option("escape", "\"").csv('../datasets/imdb/test_extra_data.csv', header=True,
    inferSchema=True)

In [142]:
from pyspark.sql.functions import col, udf, when, split, regexp_replace
from pyspark.sql.types import BooleanType, IntegerType

import re

def manage_awards(award):

    if award is not None:
        if re.search('win', award) or re.search('wins', award) or re.search('won', award):
            return True
        else:
            return False
    else:
        return False

my_udf = udf(manage_awards, BooleanType())

def preprocessing_method(df):

    """
    This method will convert to integer all the years and will put to categorical weather
    a movie won any awards or not.
    :param df: dataframe
    :return: return a preprocessed dataframe
    """

    """
    Here casting to integer all the numerical values
    """
    df = df.withColumn('startYear', col('startYear').cast('int'))
    df = df.withColumn('endYear', col('endYear').cast('int'))
    df = df.withColumn('numVotes', col('numVotes').cast('int'))

    df = df.withColumn("boxoffice", regexp_replace(col("boxoffice"), "[$,]", "").cast("int"))
    df = df.withColumn('imdb_votes', regexp_replace(col('imdb_votes'), "[,]", "").cast('int'))

    """
    Awards transformed into True or False
    """

    df = df.withColumn('awards', my_udf(df['awards']))

    """
    Substitute numVotes value when null with the value of imdb votes
    """

    df = df.withColumn('numVotes', when(col('numVotes').isNull(), col('imdb_votes')).
                       otherwise(col('numVotes')))

    """
    Transformations from string to lists
    """

    df = df.withColumn('genre', split(col('genre'), ','))
    df = df.withColumn('actors', split(col('actors'), ','))
    df = df.withColumn('language', split(col('language'), ','))

    return df

In [143]:
"""
Preprocessing the training dataset.
"""

training_prep = preprocessing_method(training)

In [144]:
"""
Preprocessing the validation dataset.
"""

validation_prep = preprocessing_method(validation)

In [145]:
"""
Preprocessing the test dataset.
"""

test_prep = preprocessing_method(test)

In [146]:
"""
Saving to csv the training dataset.
"""

training_prep.toPandas().to_csv('../datasets/models_dataset/train.csv', index=False)

In [147]:
"""
Saving to csv the validation dataset.
"""

validation_prep.toPandas().to_csv('../datasets/models_dataset/validation.csv', index=False)


In [148]:
"""
Saving to csv the test dataset.
"""

test_prep.toPandas().to_csv('../datasets/models_dataset/test.csv', index=False)